In [1]:
!wget 'https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0' positive.csv
!wget 'https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0' negative.csv

--2019-04-27 07:57:50--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2019-04-27 07:57:51--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc63635798d6f4b8d4e3ffeca4b.dl.dropboxusercontent.com/cd/0/inline/AfyYLPEpTpIxEXM4MwohrvyXf6yP2T9Ss0mXKovPwUPQTIG9biz1aOSu54l7BNc9AbGetX3iE0TuoOiMLvLiWk0_UZgPv-PnYkA2xZhBGRp3cAIngdy2ng6N0eCX9fcx2oM/file# [following]
--2019-04-27 07:57:51--  https://ucc63635798d6f4b8d4e3ffeca4b.dl.dropboxusercontent.com/cd/0/inline/AfyYLPEpTpIxEXM4MwohrvyXf6yP2T9Ss0mXKovPwUPQTIG9biz1aOSu54l7BNc9AbGetX3iE0TuoOiMLvLiWk0_UZgPv-PnYkA2xZhBGRp3cAIngdy2ng6N0eCX9fcx2oM/fi

In [2]:
import pandas as pd

In [3]:
positive = pd.read_csv('positive.csv?dl=0',header=None,encoding='utf-8',delimiter=';')
negative = pd.read_csv('negative.csv?dl=0',header=None,encoding='utf-8',delimiter=';')

In [4]:
positive=positive[[positive.columns[3]]]
negative=negative[[negative.columns[3]]]

In [5]:
positive['target']=1
negative['target']=0

In [6]:
positive.columns=['text','target']
negative.columns=['text','target']

In [7]:
positive.head()

,text,target
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [8]:
full_set = pd.concat([positive,negative])

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

In [10]:
x_train,x_test,y_train,y_test = train_test_split(full_set['text'],full_set.target.values,test_size=0.2,shuffle=True,random_state=2121)

In [11]:
clf = make_pipeline(TfidfVectorizer(ngram_range=(1,2)),MultinomialNB())
clf.fit(x_train,y_train)
preds = clf.predict(x_test)

In [12]:
accuracy_score(y_test,preds)

0.7739546366301496

## Теперь опробуем на этих же данных простейшую сеть с предобученными ембедингами

In [14]:
!wget http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.vec

--2019-04-27 08:01:17--  http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.vec
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2767469831 (2.6G) [application/octet-stream]
Saving to: 'ft_native_300_ru_twitter_nltk_word_tokenize.vec'

ft_native_300_ru_tw 100%[===================>]   2.58G  2.94MB/s    in 15m 14s 

2019-04-27 08:16:32 (2.89 MB/s) - 'ft_native_300_ru_twitter_nltk_word_tokenize.vec' saved [2767469831/2767469831]



In [15]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding
from tqdm import tqdm_notebook

/Users/i350114/Documents/python_env/pyth3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [16]:
f = open('ft_native_300_ru_twitter_nltk_word_tokenize.vec')
embedding_values = {}
for line in tqdm_notebook(f):
    value = line.replace('\n','').split(' ')
    word = value[0]
    coef = np.array(value[1:-1],dtype = 'float32')
    embedding_values[word]=coef

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(full_set['text'])
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_test = tokenizer.texts_to_sequences(x_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 291376 unique tokens.


In [25]:
embedding_matrix = np.zeros((len(word_index)+1, 100),dtype=np.float32)
sum_finding=0
for word, i in tqdm_notebook(word_index.items()):
    try:
        embedding_vector = embedding_values[word]
        sum_finding=sum_finding+1
    except:
        embedding_vector = embedding_values["unknown"]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

HBox(children=(IntProgress(value=0, max=291376), HTML(value='')))

In [27]:
print('Нашли векторов - ',sum_finding)

Нашли векторов -  147597


In [30]:
from __future__ import print_function
import numpy as np
np.random.seed(1337) 

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, SpatialDropout1D
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU, Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop,Adam

max_features = 20000
maxlen = 40  
batch_size = 256 

print('Готовим модель...')
model = Sequential()
model.add(Embedding(
    len(word_index)+1,
    100,
    weights=[embedding_matrix],
    input_length=maxlen,
    trainable=False
))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(GRU(32)))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-3,clipnorm=5,clipvalue=5),
              metrics=['accuracy'])

X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

print('Обучаем...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=25,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Тест score:', score)
print('Тест accuracy:', acc)

Готовим модель...
Обучаем...
Train on 181467 samples, validate on 45367 samples
Epoch 1/25
181467/181467 [==============================] - 33s 182us/sample - loss: 0.6468 - accuracy: 0.6087 - val_loss: 0.6247 - val_accuracy: 0.6318
Epoch 2/25
181467/181467 [==============================] - 32s 176us/sample - loss: 0.6230 - accuracy: 0.6310 - val_loss: 0.6149 - val_accuracy: 0.6389
Epoch 3/25
181467/181467 [==============================] - 32s 174us/sample - loss: 0.6153 - accuracy: 0.6380 - val_loss: 0.6115 - val_accuracy: 0.6418
Epoch 4/25
181467/181467 [==============================] - 31s 171us/sample - loss: 0.6108 - accuracy: 0.6448 - val_loss: 0.6122 - val_accuracy: 0.6392
Epoch 5/25
181467/181467 [==============================] - 31s 171us/sample - loss: 0.6066 - accuracy: 0.6482 - val_loss: 0.6071 - val_accuracy: 0.6448
Epoch 6/25
181467/181467 [==============================] - 31s 169us/sample - loss: 0.6038 - accuracy: 0.6514 - val_loss: 0.5986 - val_accuracy: 0.6593
Ep

KeyboardInterrupt: 